## Vorhersagemodellmetriken für unausgeglichene Klassifizierung

Es ist sehr üblich, dass das Accuracy oft das erste Maß ist, das wir verwenden, wenn wir Modelle für unsere Klassifizierungsprobleme bewerten.

In unserem Fall wird, aus den von uns verarbeiteten Modellen, das KNeighborsClassifier-Model mit dem besten Accuracy (accuracy_score: 89.09%) ausgewählt.
In der Realität ist das Accuracy jedoch nicht als Kriterium für die Auswahl eines guten Vorhersagemodells für unseren Datenset geeignet, weil die Classification sehr unausgewogen ist.


In [ ]:
# Untersuchung für target column 'IsBadBuy'
print(pd.crosstab(target_test, columns = 'count' ))    # normalize = True
print(target_test.shape)

In [ ]:
                                                                                # normalize = True
                                                                                
col_0     count                                                                 col_0        count
IsBadBuy                                                                        IsBadBuy
0          5716                                                                 0         0.871076
1           846                                                                 1         0.128924
(6562,)                                                                         (6562,)

class 0 : class 1 = 87 % : 13 %

Das Accuracy Paradox ist die genaue Benennung der Situation unserer Classification.
Es ist der Fall, wo wir das ausgezeichnete Accuracy haben (z. B. 89 %), aber es spiegelt meistens nur die majority class(class 0) wider.


In diesem Zusammenhang ist der f1-Score mehr für ein unausgeglichenes Klassifizierungsproblem geeignet als für das Accuracy.   Wir konzentrieren uns auf die f1-score und ich werde versuchen, mehr über den f-Beta-Score herauszufinden und auch den ROC_AOC Score damit zu vergleichen. Dies wird uns helfen, unsere Klassifizierung genauer zu analysieren und zu verstehen.


Wenn wir das Szenario betrachten, können wir sehen, welche Werte wichtiger und welche weniger wichtig sind.

Szenario:

In [ ]:
Ein Fehlkauf führt neben den Anschaffungskosten zu weiteren Kosten, wie der Einlagerung und Reparatur des Wagens. 
Deshalb ist es deiner Chefin wichtig, möglichst viele Fehlkäufe auszuschließen. Das darf allerdings nicht dazu führen, dass zu viele gute Käufe ausgeschlossen werden. 
Genauere Angaben zu den Kosten und Gewinnen der jeweiligen Käufe erhältst du für die Entwicklung des Prototyps noch nicht.

Das ‘**möglichst viele Fehlkäufe auszuschließen’** bedeutet, dass TP so groß wie möglich ist (oder FN so klein wie möglich ist).
Das ‘**nicht zu viele gute Käufe ausgeschlossen werden’**   bedeutet, dass FP so klein wie möglich ist

* Comfusion Matrix 

    TP : True Positive vom Confusion Matrix

    FP : False Positive vom Confusion Matrix

<img style="float: center;" width ="400" src="confusionmatrix01.png" />

Wir haben RandomForestClassifier, KNeighborsClassifier und LogisticRegression als Vorhersagemodelle verwendet, und die Bewertung wurde sowohl von f1 als auch von roc_aoc durchgeführt.

Daraus lässt sich schließen, dass insgesamt sechs Modelle verglichen und analysiert worden sind.


#### Python Code  : Modelle erstellen

In [ ]:
# RandomForestClassifier
search_space_rf= {'max_depth': np.geomspace(start=1, stop=50, num=10, dtype='int'),
                  'min_samples_leaf': np.geomspace(start=1, stop=500, num=10, dtype='int')}

model_rf = GridSearchCV(estimator=RandomForestClassifier(n_estimators=10, class_weight= 'balanced',random_state=42, n_jobs=-1 ),   
                            param_grid=search_space_rf ,               
                            scoring='f1',                   # scoring='roc_auc'  
                            cv=5,                       
                            n_jobs=-1)                    
model_rf.fit(features_train_selected, final_target_train)



##  LogisticRegression
pipeline_log  = Pipeline([('scaler', StandardScaler()), 
                          ('log', LogisticRegression(solver='lbfgs', max_iter=1e4, class_weight= 'balanced', n_jobs=-1))])
C_values = np.geomspace(0.001, 1000, 14)

search_space_log = [{'log__penalty' : ['l1', 'l2'], 'log__C':C_values }]
model_log = GridSearchCV(estimator=pipeline_log,   
                            param_grid=search_space_log ,               
                            scoring='f1',                  # scoring='roc_auc'  
                            cv=5,                        
                            n_jobs=-1)                    
model_log.fit(features_train_selected, final_target_train)



# KNeighborsClassifier 
pipeline_std_knn = Pipeline([('std', StandardScaler()), 
                             ('knn', KNeighborsClassifier(n_jobs=-1))])

k = np.geomspace(1, 200, 5, dtype='int')
k = np.unique(k)
search_space = {'knn__n_neighbors': k, 'knn__weights': ['uniform']}

model_grid_knn = GridSearchCV(estimator=pipeline_std_knn,  
                                 param_grid=search_space ,              
                                 scoring='f1',                  # scoring='roc_auc'  
                                 cv=5,                   
                                 n_jobs=-1)                    
model_grid_knn.fit(features_train_selected,final_target_train)

#### Ergebnis :

In [ ]:
model_rf                                                 model_grid_knn                                           model_log
**************************************                   **************************************                   **************************************
accuracy_score: 84.09%                                   accuracy_score: 89.09%                                   accuracy_score: 75.54%
f05_score: 38.34%                                        f05_score: 50.80%                                        f05_score: 32.21%
f1_score: 38.37%                                         f1_score: 33.08%                                         f1_score: 39.14%
f2_score: 38.40%                                         f2_score: 24.53%                                         f2_score: 49.86%
recall_score: 38.42%                                     recall_score: 20.92%                                     recall_score: 60.99%
precision_score: 38.33%                                  precision_score: 79.02%                                  precision_score: 28.81%
roc_auc_score: 64.63%                                    roc_auc_score: 60.05%                                    roc_auc_score: 69.34%




              precision  recall  f1-score  support                  precision  recall  f1-score  support                  precision  recall  f1-score  support

           0       0.91    0.91      0.91     5714               0       0.99    0.89      0.94     6338               0       0.78    0.93      0.85     4771
           1       0.38    0.38      0.38      848               1       0.21    0.79      0.33      224               1       0.61    0.29      0.39     1791

    accuracy                         0.84     6562        accuracy                         0.89     6562        accuracy                         0.76     6562
   macro avg       0.65    0.65      0.65     6562       macro avg       0.60    0.84      0.64     6562       macro avg       0.69    0.61      0.62     6562
weighted avg       0.84    0.84      0.84     6562    weighted avg       0.97    0.89      0.92     6562    weighted avg       0.73    0.76      0.72     6562

<img style="float: center;" src="confusionmatrix02.png" />

#### Anwendungsvorschlag für F Beta Measure und Accuracy

- **Sind Sie Klassenlabels vorhergesagt?**
    - **Ist die positive Klasse(1) wichtiger?**
        - **Sind falsch negative und falsch positive Ergebnisse gleich wichtig?**
            - F1-Mesaure verwenden
        - **Sind falsch negative Ergebnisse wichtiger?**
            - Verwenden Sie F2-Measure
        - **Sind falsch positive Ergebnisse wichtiger?**
            - F0.5-Mesaure verwenden
            
    - **Sind beide Klassen wichtig?**
        - **Haben Sie weniger als 80% Beispiele für die Mehrheitsklasse?**
            - Accuracy verwenden
        - **Haben Sie mehr 80% Beispiele für die Mehrheitsklasse?**
            - G-Mean verwenden

            


#### Metrics Vergleichen und  das best geeignetste Modell auswählen

* Metrics Vergleichen

Die Genauere Angaben zu den Kosten und Gewinnen der jeweiligen Käufe erhalten wir nicht.

Wir könnten zunächst  annehmen, dass die positive Klasse(1) wichtiger  als die 0 Klasse ist.
und nehmen wir an, die falsch negativen und falsch positiven Ergebnisse sind gleich wichtig.   
    -  der durchschnittliche Verlust durch einen falschen Kauf und der durchschnittliche Gewinn durch einen guten Kauf sind gleich

    d.H. wir vergleichen die F1 Score  und überprüfen welches Model die höchste Bewertung hat.

In [ ]:
1. **F1 Score**  
    
    model_log und model_log_roc haben beide den höchsten f1-Wert **39.14 %**

wieder aus unserem Szenario

Das ‘**möglichst viele Fehlkäufe auszuschließen’** bedeutet, dass TP so groß wie möglich ist (oder FN so klein wie möglich ist)
das ‘**nicht zu viele gute Käufe ausgeschlossen werden’**   bedeutet, dass FP so klein wie möglich ist

TP : True Positive vom Confusion Matrix  
FP : False Positive vom Confusion Matrix

daher nehmen wir jetzt an, dass die positive Klasse wichtiger und gleichzeitig die FN wichtiger als FP ist. 

    d.H. wir vergleichen die F2 Score  und überprüfen welches Model die höchste Bewertung hat.

In [ ]:
2. **F2 Score**  
    
    model_log und model_log_roc haben beide den höchsten f2-Wert **49.86 %**

In [ ]:
3. **ROC_AOC** 
    
    model_log und model_log_roc hat  den höchsten ROC_AOC-Wert **69.34 %**
    


#### Modellauswahl

    Die LogisticRegression  wird gemäß den drei oben berechneten Metrikwerten als das beste Modell ausgewählt.

#### Überlegung 
- Wenn der durchschnittliche Gewinn durch einen guten Kauf viel größer als  der Verlust von einem falschen Kauf ist,  werden wir den F0.5 Score bevorzugen.
- in dem Fall model_grid_knn hat den höchsten F0.5-Wert **50.80%**